In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import tensorflow as tf
import time

In [3]:
print(tf.__version__)

2.0.0


# A brief summary of major changes

- API cleanup. Removes redundant APIs, makes APIs more consistent.
- Eager execution. Decorate a Python function using `tf.function()` to mark it for JIT compilation.
- No more "globals". If you lose track of a `tf.Variable`, it gets garbage collected.

See https://www.tensorflow.org/guide/effective_tf2 for details.

# Data pipeline

You can use NumPy array or `tf.data` API for data pipelining. Generally, for larger dataset, you want your data as a `tf.data.Dataset` object. A Dataset object can be **created** from data in memory or disk, and can be **transformed** to another Dataset.

See https://www.tensorflow.org/guide/data for details.

In [4]:
# Load MNIST data from `tf.keras.datasets`.
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()  # These are NumPy arrays.

# Standardize data.
x_train = x_train / 255.0
x_test = x_test / 255.0

# Add a channel dimension.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [5]:
# Create dataset, then shuffle and batch them.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# Model

Model architecture APIs (High-level to low-level):
- Sequential model. Data goes through a sequence of layers.
- Functional API. More flexible than Sequential model.
- Layer subclassing. Subclass `tf.keras.layers.Layer` to create custom layer (custom computation blocks).
- Model subclassing. Subclass `tf.keras.Model`. Like layer subclassing, but allow you to use 

## Sequential

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
], name='mnist_sequential')

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Model summary

In [7]:
model.summary()

Model: "mnist_sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


Callbacks

In [8]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]

Train from NumPy array

In [9]:
start = time.time()
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), callbacks=callbacks)
print(f'Time taken : {time.time() - start} sec')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 97us/sample - loss: 0.2962 - accuracy: 0.9146 - val_loss: 0.1458 - val_accuracy: 0.9562
Epoch 2/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.1428 - accuracy: 0.9574 - val_loss: 0.1015 - val_accuracy: 0.9706
Epoch 3/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.1079 - accuracy: 0.9669 - val_loss: 0.0846 - val_accuracy: 0.9723
Epoch 4/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0877 - accuracy: 0.9732 - val_loss: 0.0798 - val_accuracy: 0.9761
Epoch 5/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.0748 - accuracy: 0.9758 - val_loss: 0.0720 - val_accuracy: 0.9760
Time taken : 26.533419609069824 sec


Train from `tf.data.Dataset`

In [12]:
start = time.time()
model.fit(train_dataset, epochs=5, validation_data=test_dataset)
print(f'Time taken : {time.time() - start} sec')

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0652 - accuracy: 0.9790 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0584 - accuracy: 0.9812 - val_loss: 0.0751 - val_accuracy: 0.9772
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0539 - accuracy: 0.9818 - val_loss: 0.0753 - val_accuracy: 0.9783
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0486 - accuracy: 0.9839 - val_loss: 0.0708 - val_accuracy: 0.9791
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0441 - accuracy: 0.9852 - val_loss: 0.0719 - val_accuracy: 0.9782
Time taken : 32.23463726043701 sec


Evaluate

In [13]:
model.evaluate(test_dataset)

313/313 [==============================] - 1s 2ms/step - loss: 0.0719 - accuracy: 0.9782


[0.07191426173503192, 0.9782]

Predict

In [14]:
y_pred = model.predict(x_test)

## Functional

In [15]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_functional')

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "mnist_functional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [17]:
start = time.time()
model.fit(train_dataset, epochs=5, validation_data=test_dataset)
print(f'Time taken : {time.time() - start} sec')

Epoch 1/5
1875/1875 [==============================] - 8s 5ms/step - loss: 0.2891 - accuracy: 0.9158 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1515 - accuracy: 0.9558 - val_loss: 0.1238 - val_accuracy: 0.9660
Epoch 3/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1243 - accuracy: 0.9644 - val_loss: 0.1076 - val_accuracy: 0.9699
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1100 - accuracy: 0.9697 - val_loss: 0.0997 - val_accuracy: 0.9729
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0985 - accuracy: 0.9733 - val_loss: 0.1082 - val_accuracy: 0.9732
Time taken : 34.17536807060242 sec


### Non-sequential example : ResNet

In [18]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
block_1_output = tf.keras.layers.MaxPooling2D(3)(x)

x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.add([x, block_1_output])

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='resnet_functional')

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [19]:
model.summary()

Model: "resnet_functional"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 26, 26, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 24, 24, 64)   18496       conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 8, 8, 64)     0           conv2d_3[0][0]                   
__________________________________________________________________________________

In [20]:
start = time.time()
model.fit(train_dataset, epochs=5, validation_data=test_dataset)
print(f'Time taken : {time.time() - start} sec')

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.1305 - accuracy: 0.9593 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0477 - accuracy: 0.9856 - val_loss: 0.0324 - val_accuracy: 0.9892
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0381 - accuracy: 0.9882 - val_loss: 0.0317 - val_accuracy: 0.9886
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0338 - accuracy: 0.9903 - val_loss: 0.0254 - val_accuracy: 0.9915
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0298 - accuracy: 0.9919 - val_loss: 0.0256 - val_accuracy: 0.9913
Time taken : 57.18211579322815 sec


## Layer subclassing

Create a custom "layer"

In [21]:
class ResidualBlock(tf.keras.layers.Layer):
    
    def __init__(self):
        super().__init__()  # Initialize base class.
        self.conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x += inputs
        return x

In [22]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(3)(x)

x = ResidualBlock()(x)

x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='custom_layer_functional')

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [25]:
# Now the model looks "sequential".
model.summary()

Model: "custom_layer_functional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
residual_block (ResidualBloc (None, 8, 8, 64)          73856     
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 4096)  

## Model subclassing

In [26]:
class CustomModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()  # Initialize base class.
        
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.pool = tf.keras.layers.MaxPooling2D(3)
        self.residual = ResidualBlock()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(10, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.residual(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.flatten(x)
        x = self.dense(x)
        
        return x
    
model = CustomModel()
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [27]:
start = time.time()
model.fit(train_dataset.take(30), epochs=5, validation_data=test_dataset.take(30))
print(f'Time taken : {time.time() - start} sec')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/5
30/30 [==============================] - 1s 38ms/step - loss: 1.2511 - accuracy: 0.5792 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
30/30 [==============================] - 0s 15ms/step - loss: 0.5094 - accuracy: 0.8635 - val_loss: 0.4025 - val_accuracy: 0.8760
Epoch 3/5
30/30 [==============================] - 0s 16ms/step - loss: 0.3530 - accuracy: 0.8938 - val_loss: 0.3467 - val_accuracy: 0.8927
Epoch 4/5
30/30 [==============================] - 0s 15ms/step - loss: 0.2526 -

# Training

Training APIs (High-level to low-level):
- Built-in training loops. (`model.compile(...)`, then `model.fit(...)`).
- Writing training loops from scratch with `tf.GradientTape`.

## Custom training

In [ ]:
# Load MNIST data from `tf.keras.datasets`.
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()  # These are NumPy arrays.

# Standardize data.
x_train = x_train / 255.0
x_test = x_test / 255.0

# Add a channel dimension.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Create dataset, then shuffle and batch them.
BUFFER = 1000
BATCH_SIZE = 300
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BUFFER).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    
    def __init__(self):
        super().__init__()  # Initialize base class.
        self.conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x += inputs
        return x
    
class CustomModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()  # Initialize base class.
        
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.pool = tf.keras.layers.MaxPooling2D(3)
        self.residual = ResidualBlock()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(10, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.residual(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.flatten(x)
        x = self.dense(x)
        
        return x

Functions to train and test one batch. Note that we're decorating the functions with `tf.function` to mark them for JIT compilations. We also create a function to perform the training loop.

In [ ]:
# @tf.function
def train_on_batch(x, y):
    """Train one batch of (x, y)"""
    # Compute loss while recording the gradient.
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = loss(y, y_pred)
        
    # Get gradient of weights w.r.t. loss.
    grad = tape.gradient(loss_value, model.trainable_weights)
    # Using optimizer, apply gradients to trainable weights.
    optimizer.apply_gradients(zip(grad, model.trainable_weights))
    
    # Compute metrics. Metrics will accumulate values.
    for metric in metrics:
        metric(y, y_pred)
        
    # Record loss
    loss_metric(loss_value)
        
# @tf.function
def test_on_batch(x, y):
    """Test one batch of (x, y)"""
    # Compute loss.
    y_pred = model(x, training=False)
    loss_value = loss(y, y_pred)
    
    # Compute metrics. Metrics will accumulate values.
    for metric in metrics:
        metric(y, y_pred)
        
    # Record loss
    loss_metric(loss_value)
    
def train(train_data, epochs, validation_data=None):
    """Perform training loop."""
    for epoch in range(epochs):
        
        start = time.time()
        message = []
        loss_metric_message = []
        message.append(f'Epoch {epoch}/{epochs}')
        
        # Iterate through training dataset and
        # train model on each batch.
        for i, (x, y) in enumerate(train_data):
            s = time.time()
            train_on_batch(x, y)
            print(time.time()-s)
            
            
        # Obtain metric values after trained on all batches.
        train_metric_values = {metric.name: metric.result().numpy() for metric in metrics}
        train_loss_values = {'loss': loss_metric.result().numpy()}
        # Reset metric states at the end of each epoch.
        for metric in metrics:
            metric.reset_states()
        loss_metric.reset_states()
        
        loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in train_loss_values.items()]))
        loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in train_metric_values.items()]))
        
        if validation_data is not None:
            
            # Iterate through validation dataset and
            # train model on each batch.
            for i, (x, y) in enumerate(validation_data):
                test_on_batch(x, y)

            # Obtain metric values after trained on all batches.
            val_metric_values = {f'val_{metric.name}': metric.result().numpy() for metric in metrics}
            val_loss_values = {'val_loss': loss_metric.result().numpy()}
            # Reset metric states at the end of each epoch.
            for metric in metrics:
                metric.reset_states()
            loss_metric.reset_states()
                
            loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in val_loss_values.items()]))
            loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in val_metric_values.items()]))
        
        message.append(f'{(time.time() - start):.2f} sec')
        message += loss_metric_message
        message = ' - '.join(message)
        print(message)

In [ ]:
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3)
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_metric = tf.keras.metrics.Mean('loss')

model = CustomModel()

In [ ]:
# train(train_dataset.take(30), epochs=5, validation_data=test_dataset.take(30))
train(train_dataset, epochs=5, validation_data=test_dataset)

```
WARNING:tensorflow:Layer custom_model_20 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0/5 - 9.96 sec - loss: 0.1264 - sparse_categorical_accuracy: 0.9621 - val_loss: 0.0426 - val_sparse_categorical_accuracy: 0.9855
Epoch 1/5 - 9.03 sec - loss: 0.0457 - sparse_categorical_accuracy: 0.9862 - val_loss: 0.0319 - val_sparse_categorical_accuracy: 0.9898
Epoch 2/5 - 9.09 sec - loss: 0.0372 - sparse_categorical_accuracy: 0.9894 - val_loss: 0.0259 - val_sparse_categorical_accuracy: 0.9911
Epoch 3/5 - 9.06 sec - loss: 0.0327 - sparse_categorical_accuracy: 0.9910 - val_loss: 0.0263 - val_sparse_categorical_accuracy: 0.9908
Epoch 4/5 - 9.12 sec - loss: 0.0290 - sparse_categorical_accuracy: 0.9916 - val_loss: 0.0239 - val_sparse_categorical_accuracy: 0.9923
```

### Test - eager execution

In [ ]:
def train_on_batch(x, y):
    """Train one batch of (x, y)"""
    # Compute loss while recording the gradient.
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = loss(y, y_pred)
        
    # Get gradient of weights w.r.t. loss.
    grad = tape.gradient(loss_value, model.trainable_weights)
    # Using optimizer, apply gradients to trainable weights.
    optimizer.apply_gradients(zip(grad, model.trainable_weights))
    
    # Compute metrics. Metrics will accumulate values.
    for metric in metrics:
        metric(y, y_pred)
        
    # Record loss
    loss_metric(loss_value)
        
def test_on_batch(x, y):
    """Test one batch of (x, y)"""
    # Compute loss.
    y_pred = model(x, training=False)
    loss_value = loss(y, y_pred)
    
    # Compute metrics. Metrics will accumulate values.
    for metric in metrics:
        metric(y, y_pred)
        
    # Record loss
    loss_metric(loss_value)
    
def train(train_data, epochs, validation_data=None):
    """Perform training loop."""
    for epoch in range(epochs):
        
        start = time.time()
        message = []
        loss_metric_message = []
        message.append(f'Epoch {epoch}/{epochs}')
        
        # Iterate through training dataset and
        # train model on each batch.
        for i, (x, y) in enumerate(train_data):
            train_on_batch(x, y)
            
        # Obtain metric values after trained on all batches.
        train_metric_values = {metric.name: metric.result().numpy() for metric in metrics}
        train_loss_values = {'loss': loss_metric.result().numpy()}
        # Reset metric states at the end of each epoch.
        for metric in metrics:
            metric.reset_states()
        loss_metric.reset_states()
        
        loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in train_loss_values.items()]))
        loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in train_metric_values.items()]))
        
        if validation_data is not None:
            
            # Iterate through validation dataset and
            # train model on each batch.
            for i, (x, y) in enumerate(validation_data):
                test_on_batch(x, y)

            # Obtain metric values after trained on all batches.
            val_metric_values = {f'val_{metric.name}': metric.result().numpy() for metric in metrics}
            val_loss_values = {'val_loss': loss_metric.result().numpy()}
            # Reset metric states at the end of each epoch.
            for metric in metrics:
                metric.reset_states()
            loss_metric.reset_states()
                
            loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in val_loss_values.items()]))
            loss_metric_message.append(' - '.join([f'{k}: {v:.4f}' for k,v in val_metric_values.items()]))
        
        message.append(f'{(time.time() - start):.2f} sec')
        message += loss_metric_message
        message = ' - '.join(message)
        print(message)

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_metric = tf.keras.metrics.Mean('loss')

model = CustomModel()

In [ ]:
# train(train_dataset.take(30), epochs=5, validation_data=test_dataset.take(30))
train(train_dataset, epochs=5, validation_data=test_dataset)

```
WARNING:tensorflow:Layer custom_model_21 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0/5 - 41.06 sec - loss: 0.1298 - sparse_categorical_accuracy: 0.9601 - val_loss: 0.0408 - val_sparse_categorical_accuracy: 0.9854
Epoch 1/5 - 40.79 sec - loss: 0.0468 - sparse_categorical_accuracy: 0.9859 - val_loss: 0.0433 - val_sparse_categorical_accuracy: 0.9873
Epoch 2/5 - 41.14 sec - loss: 0.0384 - sparse_categorical_accuracy: 0.9889 - val_loss: 0.0346 - val_sparse_categorical_accuracy: 0.9892
Epoch 3/5 - 40.74 sec - loss: 0.0320 - sparse_categorical_accuracy: 0.9910 - val_loss: 0.0332 - val_sparse_categorical_accuracy: 0.9896
Epoch 4/5 - 40.80 sec - loss: 0.0303 - sparse_categorical_accuracy: 0.9915 - val_loss: 0.0370 - val_sparse_categorical_accuracy: 0.9897
```

### Test - v1 graph

In [ ]:
import numpy as np
y_train_ohe = np.eye(10)[y_train]
y_test_ohe = np.eye(10)[y_test]

In [ ]:
def test_tf1():
    
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    
    epochs = 5
    BATCH_SIZE=1000

    tf.reset_default_graph()

    tr_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train_ohe)).shuffle(1000).batch(BATCH_SIZE)
    va_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test_ohe)).batch(BATCH_SIZE)

    iterator = tf.data.Iterator.from_structure(
        tr_dataset.output_types,
        tr_dataset.output_shapes)

    tr_initializer = iterator.make_initializer(tr_dataset)
    va_initializer = iterator.make_initializer(va_dataset)

    training = tf.placeholder_with_default(input=False, shape=(), name='training')

    x, y = iterator.get_next()
    x = tf.layers.conv2d(x, 32, 3, activation='relu')
    x = tf.layers.conv2d(x, 64, 3, activation='relu')
    x1 = tf.layers.max_pooling2d(x, 3, 3)

    x = tf.layers.conv2d(x1, 64, 3, activation='relu', padding='same')
    x = tf.layers.conv2d(x, 64, 3, activation='relu', padding='same')
    x = tf.add(x, x1)

    x = tf.layers.flatten(x)
    x = tf.layers.dropout(x, 0.5, training=training)
    logits = tf.layers.dense(x, 10)
    probs = tf.nn.softmax(logits)
    classes = tf.argmax(probs, axis=1)
    true_classes = tf.argmax(y, axis=1)

    loss = tf.nn.softmax_cross_entropy_with_logits_v2(y, logits)
    acc = tf.metrics.accuracy(true_classes, classes)
#     opt = tf.train.AdamOptimizer().minimize(loss)
    opt = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

    start = time.time()
    with tf.Session() as sess:

        print(f'Init time: {time.time() - start}')

        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())

        for epoch in range(epochs):

            start = time.time()

    #         s = time.time()
            sess.run(tr_initializer)
    #         e = time.time()
    #         print(f'tr init - {e-s}')

    #         s = time.time()
            ls = []
            accs = []
            while True:
                try:
                    _, l, a = sess.run([opt, loss, acc], feed_dict={training:True})
                    ls.append(np.mean(l))
                    accs.append(np.mean(a))
                except Exception as e:
                    break
            tl = np.mean(ls)
            ta = np.mean(accs)
    #         e = time.time()
    #         print(f'tr loop - {e-s}')

    #         s = time.time()
            sess.run(va_initializer)
    #         e = time.time()
    #         print(f'va init - {e-s}')

    #         s = time.time()
            ls = []
            accs = []
            while True:
                try:
                    l, a = sess.run([loss, acc], feed_dict={training:False})
                    ls.append(np.mean(l))
                    accs.append(np.mean(a))
                except Exception:
                    break
            vl = np.mean(ls)
            va = np.mean(accs)
    #         e = time.time()
    #         print(f'va loop - {e-s}')

            print(f'Epoch {epoch}/{epochs} - {(time.time()-start):.2f} sec - loss: {tl:.4f} - accuracy: {ta:.4f} - val_loss: {vl:.4f} - val_accuracy: {va:.4f}')
    
test_tf1()